In [579]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmm.app import element as E
from math import ceil
import os
import sys
import mdtraj as md
import numpy as np
import pandas as pd
import csv
import pprint
import matplotlib.pyplot as plt
import random
import copy

In [580]:
pdb_filepath = "./alanine-dipeptide-nowater.pdb"
ff_filepath  = "./GB99dms.xml"

In [581]:
pdb = PDBFile(pdb_filepath)
ff = ForceField(ff_filepath)
top = pdb.topology

In [139]:
nonbondedMethod=CutoffNonPeriodic
dt = 0.004 # ps
n_steps       = int(ceil(2e6 / dt)) # 2 μs
n_steps_equil = int(ceil(10 / dt)) # 500 ps
n_steps_save  = int(ceil(10 / dt)) # 500 ps
temperature = 300.0 * kelvin
kappa = 0.7 / nanometer
friction = 1 / picosecond
nonbondedCutoff = 2 * nanometer
restraint_fc = 500.0 # kJ/mol

system = ff.createSystem(
    pdb.topology,
    nonbondedMethod=nonbondedMethod,
    nonbondedCutoff=nonbondedCutoff,
    constraints=HBonds,
    hydrogenMass=2*amu,
    implicitSolventKappa=kappa,
)

In [140]:
ff.generateTemplatesForUnmatchedResidues(top) #力場が利用できない残基

[[], []]

In [141]:
ff.getGenerators() #全てのgeneratorをプリント

In [142]:
templates = ff.getMatchingTemplates(top,ignoreExternalBonds=False) #指定されたトポロジーの残基にマッチする力場残基テンプレートのリストを返す
for(residue, template) in zip(top.residues(), templates):
    print(f'Residue {residue.id} {residue.name} matched template {template.name}' )

Residue 1 ACE matched template ACE
Residue 2 ALA matched template ALA
Residue 3 NME matched template NME


In [143]:
ff.getUnmatchedResidues(top) #力場テンプレートが使用できない、残基のリストを返す

[]

In [144]:
ff.loadFile #指定したXMLファイルを読み込みこのFFに追加する
ff.registerAtomType #アトムタイプの追加

<bound method ForceField.registerAtomType of <openmm.app.forcefield.ForceField object at 0x7f6fc05c4b80>>

In [ ]:
class HarmonicBondGenerator(object):
    """A HarmonicBondGenerator constructs a HarmonicBondForce."""

    def __init__(self, forcefield):
        self.ff = forcefield
        self.bondsForAtomType = defaultdict(set)
        self.types1 = []
        self.types2 = []
        self.length = []
        self.k = []
    
class HarmonicAngleGenerator(object):
    """A HarmonicAngleGenerator constructs a HarmonicAngleForce."""

    def __init__(self, forcefield):
        self.ff = forcefield
        self.anglesForAtom2Type = defaultdict(list)
        self.types1 = []
        self.types2 = []
        self.types3 = []
        self.angle = []
        self.k = []

#二面角
class PeriodicTorsion(object):
    """A PeriodicTorsion records the information for a periodic torsion definition."""

    def __init__(self, types):
        self.types1 = types[0]
        self.types2 = types[1]
        self.types3 = types[2]
        self.types4 = types[3]
        self.periodicity = []
        self.phase = []
        self.k = []
        self.ordering = 'default'

class PeriodicTorsionGenerator(object):
    """A PeriodicTorsionGenerator constructs a PeriodicTorsionForce."""

    def __init__(self, forcefield):
        self.ff = forcefield
        self.proper = []
        self.improper = []
        self.propersForAtomType = defaultdict(set)

class NonbondedGenerator(object):
    """A NonbondedGenerator constructs a NonbondedForce."""

    SCALETOL = 1e-5

    def __init__(self, forcefield, coulomb14scale, lj14scale, useDispersionCorrection):
        self.ff = forcefield
        self.coulomb14scale = coulomb14scale
        self.lj14scale = lj14scale
        self.useDispersionCorrection = useDispersionCorrection
        self.params = ForceField._AtomTypeParameters(forcefield, 'NonbondedForce', 'Atom', ('charge', 'sigma', 'epsilon'))

class _AtomTypeParameters(object):
        """Inner class used to record parameter values for atom types."""
        def __init__(self, forcefield, forceName, atomTag, paramNames):
            self.ff = forcefield
            self.forceName = forceName
            self.atomTag = atomTag
            self.paramNames = paramNames
            self.paramsForType = {}
            self.extraParamsForType = {}

In [145]:
generator = copy.deepcopy(ff.getGenerators()) #ここからffのパラメータの値を参照できる
harmonic_bond_generator = copy.deepcopy(generator[0])
hormonic_angle_generator = copy.deepcopy(generator[1])
periodic_torsion_generator = copy.deepcopy(generator[2])
non_bond_generator = copy.deepcopy(generator[3])

In [146]:
#HarmonicBond
harmonic_bond_generator.types1 #types1
harmonic_bond_generator.types2 #types2
harmonic_bond_generator.length #length
harmonic_bond_generator.k[0:10]#k

[265265.6,
 284512.0,
 282001.60000000003,
 307105.6,
 462750.39999999997,
 267776.0,
 267776.0,
 259408.0,
 259408.0,
 284512.0]

In [147]:
#PeriodicTorsion
periodic_torsion_generator.proper[1].k

[-2.6944457920000002, -6.6866596000000005, -1.671106336, 0.3141874384]

In [148]:
#NonBondGenerator
non_bond_generator.params.paramsForType

{'H': {'sigma': 0.1051327626910855, 'epsilon': 0.06495320495709334},
 'HB': {'sigma': 0.10690784617684071, 'epsilon': 0.06568879999999999},
 'HO': {'sigma': 0.0, 'epsilon': 0.0},
 'HS': {'sigma': 0.10690784617684071, 'epsilon': 0.06568879999999999},
 'HC': {'sigma': 0.25861690367065504, 'epsilon': 0.06367916271616703},
 'H1': {'sigma': 0.24577349171567348, 'epsilon': 0.06558010092036579},
 'H2': {'sigma': 0.22931733004932334, 'epsilon': 0.06568879999999999},
 'H3': {'sigma': 0.2114993556865165, 'epsilon': 0.06568879999999999},
 'HP': {'sigma': 0.1964271265155058, 'epsilon': 0.0657702120349221},
 'HA': {'sigma': 0.2606875475881734, 'epsilon': 0.06298040345497975},
 'H4': {'sigma': 0.25105525877194756, 'epsilon': 0.06276},
 'H5': {'sigma': 0.2421462715905442, 'epsilon': 0.06276},
 'HW': {'sigma': 0.0, 'epsilon': 0.0},
 'HZ': {'sigma': 0.259964245953351, 'epsilon': 0.06276},
 'O': {'sigma': 0.29614478406579436, 'epsilon': 0.8627567000561271},
 'OB': {'sigma': 0.2959921901149463, 'epsilon'

In [149]:
#for example
for g in ff.getGenerators():
  if isinstance(g, app.forcefield.HarmonicBondGenerator ):
    for i in range(len(g.types1)):
      print(g.types1[i], g.types2[i], g.length[i], g.k[i])

['C'] ['C1'] 0.1522 265265.6
['C1'] ['H1'] 0.10900000000000001 284512.0
['C1'] ['N'] 0.1449 282001.60000000003
['C1'] ['N3'] 0.1471 307105.6
['HO'] ['O3'] 0.096 462750.39999999997
['C8'] ['O3'] 0.141 267776.0
['C8'] ['OH'] 0.141 267776.0
['CT'] ['C8'] 0.1526 259408.0
['CT'] ['C9'] 0.1526 259408.0
['C9'] ['HC'] 0.10900000000000001 284512.0
['C9'] ['H1'] 0.10900000000000001 284512.0
['C9'] ['S'] 0.181 189953.6
['C9'] ['C'] 0.1522 265265.6
['C8'] ['H1'] 0.10900000000000001 284512.0
['C8'] ['HC'] 0.10900000000000001 284512.0
['OW'] ['HW'] 0.09572 462750.39999999997
['HW'] ['HW'] 0.15136 462750.39999999997
['C'] ['C'] 0.1525 259408.0
['C'] ['CA'] 0.1409 392459.2
['C'] ['CB'] 0.1419 374049.60000000003
['C'] ['CM'] 0.1444 343088.0
['C'] ['CT'] 0.1522 265265.6
['C'] ['N'] 0.1335 410032.0
['C'] ['N*'] 0.1383 354803.2
['C'] ['NA'] 0.13879999999999998 349782.4
['C'] ['NC'] 0.1358 382417.6
['C'] ['O'] 0.12290000000000001 476976.0
['C'] ['O2'] 0.125 548940.8
['C'] ['OH'] 0.13640000000000002 376560.

### 二面角のパラメータ修正に用いるコード

In [150]:
nff = len(periodic_torsion_generator.proper) #対象にするパラメータ数
nff

193

In [151]:
periodic_torsion_generator

In [152]:
#create AtomType list
import itertools
atom_type_list = [] #AtomTypeを格納するリスト
for i in range(nff):
    tmp = []
    tmp.append(periodic_torsion_generator.proper[i].types1 if type(periodic_torsion_generator.proper[i].types1) == list else ['*'] )
    tmp.append(periodic_torsion_generator.proper[i].types2 if type(periodic_torsion_generator.proper[i].types2) == list else ['*'] )
    tmp.append(periodic_torsion_generator.proper[i].types3 if type(periodic_torsion_generator.proper[i].types3) == list else ['*'] )
    tmp.append(periodic_torsion_generator.proper[i].types4 if type(periodic_torsion_generator.proper[i].types4) == list else ['*'] )
    #tmp = list(itertools.chain.from_iterable(tmp)) #二次元リストを一次元に
    atom_type_list.append(tmp)

print(atom_type_list[0])

[['C'], ['N'], ['C1'], ['C']]


### openffでのatom type

In [153]:
top_atom = [atom.name for atom in top.atoms()] #topologyのatom type
templates = ff.getMatchingTemplates(top) # topologyのresidueごとのtemplateのリスト

data = ff._SystemData(top)
ff._matchAllResiduesToTemplates(data, top, ff._buildBondedToAtomList(top), templates, False)

ff_atom = []
for atom in top.atoms():
        typename = data.atomType[atom]
        ff_atom.append(typename)

print("top_atom :", top_atom)
print("ff_atom :", ff_atom)

top_atom : ['H1', 'CH3', 'H2', 'H3', 'C', 'O', 'N', 'H', 'CA', 'HA', 'CB', 'HB1', 'HB2', 'HB3', 'C', 'O', 'N', 'H', 'C', 'H1', 'H2', 'H3']
ff_atom : ['HC', 'CT', 'HC', 'HC', 'C', 'O', 'N', 'H', 'CT', 'H1', 'CT', 'HC', 'HC', 'HC', 'C', 'O', 'N', 'H', 'CT', 'H1', 'H1', 'H1']


### mdtrajからalanine_dipeptideの二面角phiとpsiのインデックスを持ってくる

In [154]:
#mdtraj
#analysis alanine_dipeptide
t= md.load('alanine-dipeptide-nowater.pdb')
top = t.topology

In [155]:
traj_target_path = 'traj_org.dcd'
traj_path = 'traj_new.dcd'
traj_target = md.load_dcd(traj_target_path, top=top)
traj = md.load_dcd(traj_path, top=top)

In [156]:
for i in range(top.n_atoms):
    print( i, top.atom(i).name)
phi_top_index = list(itertools.chain.from_iterable(md.compute_phi(traj)[0])) #toplogyでのphiのインデックス
psi_top_index = list(itertools.chain.from_iterable(md.compute_psi(traj)[0]))

print("phi_top_index :", phi_top_index)
print("psi_top_index :", psi_top_index)

phi_atom_type = []
psi_atom_type = []

for i in range(len(phi_top_index)):
    phi_atom_type.append([ff_atom[phi_top_index[i]]]) #ffでのatom type
    psi_atom_type.append([ff_atom[psi_top_index[i]]])

print("phi_atom_type :", phi_atom_type)
print("psi_atom_type :", psi_atom_type)

0 H1
1 CH3
2 H2
3 H3
4 C
5 O
6 N
7 H
8 CA
9 HA
10 CB
11 HB1
12 HB2
13 HB3
14 C
15 O
16 N
17 H
18 C
19 H1
20 H2
21 H3
phi_top_index : [4, 6, 8, 14]
psi_top_index : [6, 8, 14, 16]
phi_atom_type : [['C'], ['N'], ['CT'], ['C']]
psi_atom_type : [['N'], ['CT'], ['C'], ['N']]


## To do
atom_type_listからphi_atom_typeとpsi_atom_typeを含んでいるものを出力
ffの値を変更する
それをxmlファイルで出力して、シミュレーションを行う

In [182]:
pdb = PDBFile(pdb_filepath)
ff = ForceField(ff_filepath)
top = pdb.topology

phi_ff_index = [] #ffでphiに関係するパラメータのインデックス
psi_ff_index = [] #ffでpsiに関係するパラメータのインデックス
for i in range(len(atom_type_list)):
    if(atom_type_list[i] == phi_atom_type):
        phi_ff_index.append(i)
    if(atom_type_list[i] == psi_atom_type):
        psi_ff_index.append(i)

print(phi_ff_index)

[113]


## create ff

In [183]:
phi_original = periodic_torsion_generator.proper[phi_ff_index[0]] #ffでphiに関するtorsionの力場パラメータの行を抜き出す
psi_original = periodic_torsion_generator.proper[psi_ff_index[0]] #ffでpsiに関するtorsionの力場パラメータの行を抜き出す

#空のインスタンスを作成
phi_updated = app.forcefield.PeriodicTorsion(phi_atom_type) 
psi_updated = app.forcefield.PeriodicTorsion(psi_atom_type)

print(vars(phi_original))
print(vars(phi_updated))

{'types1': ['C'], 'types2': ['N'], 'types3': ['CT'], 'types4': ['C'], 'periodicity': [1, 2, 3, 4, 2], 'phase': [0.0, 0.0, 0.0, 0.0, 1.5707963267948966], 'k': [-0.13725408041057408, 1.3803857819285754, 2.339932407964332, 0.3283656547765962, 1.6668558389770463], 'ordering': 'default'}
{'types1': ['C'], 'types2': ['N'], 'types3': ['CT'], 'types4': ['C'], 'periodicity': [], 'phase': [], 'k': [], 'ordering': 'default'}


In [184]:
maximum_update_ratio = 0.03 #[0, maximum_update_ratio]の範囲でパラメータの値を変える
rnd = random.uniform(-maximum_update_ratio, maximum_update_ratio) #[0, maximum_update_ratio]の範囲の乱数
print(rnd)

-0.020189204705566618


In [185]:
#periodicity
phi_updated.periodicity = phi_original.periodicity
psi_updated.periodicity = psi_original.periodicity

#phase
phi_updated.phase = phi_original.phase
psi_updated.phase = psi_original.phase

In [186]:
phi_updated

In [187]:
#k
#今回はkのみ変える
for i in range(len(phi_original.k)):
    rnd = random.uniform(-maximum_update_ratio, maximum_update_ratio) #最大でamximum_update_ratioの割合でパラメータを変える
    phi_updated.k.append(phi_original.k[i] * (1 + rnd))
    
for i in range(len(psi_original.k)):
    rnd = random.uniform(-maximum_update_ratio, maximum_update_ratio)
    psi_updated.k.append(psi_original.k[i] * (1 + rnd))

In [343]:
vars(phi_updated) #辞書型

{'types1': ['C'],
 'types2': ['N'],
 'types3': ['CT'],
 'types4': ['C'],
 'periodicity': [1, 2, 3, 4, 2],
 'phase': [0.0, 0.0, 0.0, 0.0, 1.5707963267948966],
 'k': [-0.14131023094487127,
  1.3931040613985428,
  2.283760283655059,
  0.3221007267272557,
  1.7115868963324152],
 'ordering': 'default'}

In [189]:
periodic_torsion_generator.proper[phi_ff_index[0]] = phi_updated
periodic_torsion_generator.proper[psi_ff_index[0]] = psi_updated

In [190]:
generator[2] = periodic_torsion_generator #torsionに関するffをアップデート

In [191]:
updated_ff = copy.deepcopy(ff) #元のffをコピー

In [192]:
updated_ff.getGenerators()[2].proper[phi_ff_index[0]].k #値の確認

[-0.1396954369058983,
 1.4020721749174487,
 2.2722793075263983,
 0.33249158784820976,
 1.669819011164073]

In [193]:
ff.getGenerators()[2].proper[phi_ff_index[0]].k #値の確認

[-0.1396954369058983,
 1.4020721749174487,
 2.2722793075263983,
 0.33249158784820976,
 1.669819011164073]

In [194]:
updated_ff.getGenerators()[2] = generator[2] #ffをアップデート

In [195]:
updated_ff.getGenerators()[2].proper[phi_ff_index[0]].k

[-0.14131023094487127,
 1.3931040613985428,
 2.283760283655059,
 0.3221007267272557,
 1.7115868963324152]

## updated_ffをXMLファイルに記述できるように辞書型に変換

In [536]:
def convert_openmm_to_XML(openmm_dict):
    xml_dict = dict() #変換後の辞書
    nparam = len(openmm_dict['k']) #param数

    #atom_typeの定義
    xml_dict['types1'] = openmm_dict['types1'][0]
    xml_dict['types2'] = openmm_dict['types2'][0]
    xml_dict['types3'] = openmm_dict['types3'][0]
    xml_dict['types4'] = openmm_dict['types4'][0]

    #paramを定義
    for i in range(nparam):
        xml_dict[f'periodicity{i+1}'] = str(openmm_dict['periodicity'][i])
        xml_dict[f'phase{i+1}'] = str(openmm_dict['phase'][i])
        xml_dict[f'k{i+1}'] = str(openmm_dict['k'][i])
    
    return xml_dict      

In [537]:
phi_xml = convert_openmm_to_XML(vars(phi_updated))
psi_xml = convert_openmm_to_XML(vars(psi_updated))

In [587]:
phi_xml

{'types1': 'C',
 'types2': 'N',
 'types3': 'CT',
 'types4': 'C',
 'periodicity1': '1',
 'phase1': '0.0',
 'k1': '-0.14131023094487127',
 'periodicity2': '2',
 'phase2': '0.0',
 'k2': '1.3931040613985428',
 'periodicity3': '3',
 'phase3': '0.0',
 'k3': '2.283760283655059',
 'periodicity4': '4',
 'phase4': '0.0',
 'k4': '0.3221007267272557',
 'periodicity5': '2',
 'phase5': '1.5707963267948966',
 'k5': '1.7115868963324152'}

## Create XML File

## To do
- とりあえずコメントを綺麗に書く
- xmlの属性を変えられるようにする

In [573]:
import xml.etree.ElementTree as ET
tree = ET.parse(ff_filepath)
root = tree.getroot()

In [574]:
root.tag #最上位のノード

'ForceField'

In [575]:
#rootはイテレート可能な子ノードをもつ
for child in root:
    print(child.tag, child.attrib)

Info {}
AtomTypes {}
Residues {}
HarmonicBondForce {}
HarmonicAngleForce {}
PeriodicTorsionForce {'ordering': 'amber'}
NonbondedForce {'coulomb14scale': '0.8785442959154057', 'lj14scale': '0.5097903551017255'}
Script {}


In [578]:
# XMLから'PeriodicTorsionForce'を検索
for torsion in root.iter('PeriodicTorsionForce'):

    #各要素の中から、phiとpsiに関するパラメータを検索
    for ff_local in torsion:
        #print(list(ff_local.attrib.values())[0:4]) [0:4]でスライスすることで、アトムタイプを取り出す
        #phiのXMLを更新
        if list(ff_local.attrib.values())[0:4] == list(itertools.chain.from_iterable(phi_atom_type)):
            print(ff_local.attrib)
            ff_local.clear() #初期化する
            for item in phi_xml.items():
                ff_local.set(item[0], item[1])
            print(ff_local.attrib)
            print()
        
        #psiのXMLを更新
        if list(ff_local.attrib.values())[0:4] == list(itertools.chain.from_iterable(psi_atom_type)):
            print(ff_local.attrib)
            ff_local.clear() #初期化する
            for item in psi_xml.items():
                #print(item)
                ff_local.set(item[0], item[1])
            print(ff_local.attrib)
            print()

tree.write('output.xml')

{'types1': 'N', 'types2': 'CT', 'types3': 'C', 'types4': 'N', 'periodicity1': '1', 'phase1': '2.8080339837817268', 'k1': '2.7562247284353925', 'periodicity2': '1', 'phase2': '3.141592653589793', 'k2': '-0.7939603369252063', 'periodicity3': '2', 'phase3': '3.141592653589793', 'k3': '6.117627658952802', 'periodicity4': '3', 'phase4': '3.141592653589793', 'k4': '1.9447873754479845', 'periodicity5': '4', 'phase5': '3.141592653589793', 'k5': '-0.07607467117953355', 'periodicity6': '5', 'phase6': '3.141592653589793', 'k6': '-0.01603476759412381', 'periodicity7': '1', 'phase7': '1.5707963267948966', 'k7': '-1.0933094406995865', 'periodicity8': '2', 'phase8': '1.5707963267948966', 'k8': '0.31242669067675943', 'periodicity9': '3', 'phase9': '1.5707963267948966', 'k9': '0.25025337772182615', 'periodicity10': '4', 'phase10': '1.5707963267948966', 'k10': '0.10948330165418166', 'periodicity11': '5', 'phase11': '1.5707963267948966', 'k11': '-0.04510471639170056'}
{'types1': 'N', 'types2': 'CT', 'typ

In [584]:
phi_atom_type

[['C'], ['N'], ['CT'], ['C']]

In [586]:
psi_atom_type

[['N'], ['CT'], ['C'], ['N']]